# Batch ingestion into Azure Cosmos DB collection

In this notebook, we'll 

+ Load the Retail Data dataset from ADLS Gen2 to a DataFrame
+ Write the DataFrame to Azure Cosmos DB containers

>**Did you know?**  [Azure Synapse Link for Azure Cosmos DB](https://review.docs.microsoft.com/en-us/azure/cosmos-db/synapse-link?branch=release-build-cosmosdb) is a hybrid transactional and analytical processing (HTAP) capability that enables you to run near real-time analytics over operational data in Azure Cosmos DB.
&nbsp

>**Did you know?**  [Azure Cosmos DB analytical store](https://review.docs.microsoft.com/en-us/azure/cosmos-db/analytical-store-introduction?branch=release-build-cosmosdb) is a fully isolated column store for enabling large scale analytics against operational data in your Azure Cosmos DB, without any impact to your transactional workloads.
&nbsp

>**Did you know?**  The Synapse workspace is attached to an [ADLS Gen2 storage account](https://docs.microsoft.com/en-us/azure/storage/blobs/data-lake-storage-introduction) and the files placed on the default storage account can be accessed using the relative path as below.
&nbsp

>**Did you know?**  From now on, all operations are case sentitive. Please be careful with everything you need to type.
&nbsp


### 1. Using the Data / Linked tab of your Synapse workspace, one of the pre-requisites mentioned in the README file, create a **RetailData** folder within the root directory of your storage account. Upload to this folder the csv files that are placed within the folder with the same name of this repo. As you can see in the image below, your new folder in the storage account, a data lake, will be in the second level of the file system structure. 

<img src="https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/upload.PNG" alt="Upload" width="75%"/>

### 2. Using the Azure Portal, go to the **Access Control (IAM)** tab, click on the **+Add** and **Add a role assignment** links and add yourself to the **Contributor** role. This will allow you to create databases and tables within from your Azure Synapse Spark Pool.

### 3. Using the Azure Portal, go to [Data Explorer](https://docs.microsoft.com/en-us/azure/cosmos-db/data-explorer) of your the Azure Cosmos DB Account and create a database called **RetailSalesDemoDB**. Change the Throughput to Autoscale and set the limit to 40,000 instead of 400, this will speed-up the loading process of the data, scaling down the database when it is not in use. For more information, click [here](https://docs.microsoft.com/en-us/azure/cosmos-db/provision-throughput-autoscale).

### 4. In the same Data Explorer, create 3 **Analytical Store** enabled containers: **StoreDemoGraphics**, **RetailSales**, and **Products**. In the portal interface, the container-id is the container name. Important details:
+ Use **/id** as the Partition key for all 3 containers.
+ Please make sure that **Analytical store** is set to **On** for all 3 containers. 

### 5. In your Azure Synapse workspace, go to the **Manage / Linked Services** tab and create a service called **RetailSalesDemoDB** pointing to the database in Cosmos DB created in item 3 above.

<img src="https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/ls.PNG" alt="Surface Device" width="100%"/>



## 

### 6. Run the Commands below to load the data into Spark DataFrames. The Spark session will take a couple of minutes to start and your commands will be executed.

In [5]:
dfStoreDemoGraphics = (spark
                .read
                .csv("/RetailData/StoreDemoGraphics.csv", header=True)
              )

dfRetailSales = (spark
                .read
                .csv("/RetailData/RetailSales.csv", header=True)
              )

dfProduct = (spark
                .read
                .csv("/RetailData/Products.csv", header=True)
              )


### 7. Write the dataframe to the Azure Cosmos DB Collections

>**Did you know?** The "cosmos.oltp" is the Spark format that enables connection to the Cosmos DB Transactional store.

>**Did you know?** The ingestion to the Azure Cosmos DB collection is always performed through the Transactional store irrespective of whether the Analytical Store is enabled or not.

>**Did you know?** After the ingestion, Azure Cosmos DB will replicate your data from OLTP store to Analytical store in up to 5 minutes. 

In [8]:
dfStoreDemoGraphics.write\
            .format("cosmos.oltp")\
            .option("spark.synapse.linkedService", "RetailSalesDemoDB")\
            .option("spark.cosmos.container", "StoreDemoGraphics")\
            .option("spark.cosmos.write.upsertEnabled", "true")\
            .mode('append')\
            .save()

dfRetailSales.write\
            .format("cosmos.oltp")\
            .option("spark.synapse.linkedService", "RetailSalesDemoDB")\
            .option("spark.cosmos.container", "RetailSales")\
            .option("spark.cosmos.write.upsertEnabled", "true")\
            .mode('append')\
            .save()

dfProduct.write\
            .format("cosmos.oltp")\
            .option("spark.synapse.linkedService", "RetailSalesDemoDB")\
            .option("spark.cosmos.container", "Products")\
            .option("spark.cosmos.write.upsertEnabled", "true")\
            .mode('append')\
            .save()     


### 8. Using the Azure Cosmos DB Account portal, go to the Data Explorer and check if the data was loaded.

## All Done! Now let's go to the next Notebook.